# Define variables J, z, s,t

In [158]:
#REMEMBER TO ASDD ACITVATION FUCNITON!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [1]:
import numpy as np
import math

z = np.arange(0.01,0.99,1/10)
N = len(z)
J = np.arange(0,10,2.0)


In [2]:
# generate random floating point values for s and t and F
# S and T
from random import seed
from random import random
import array as arr
#random number generator for s
seed(1)
s = arr.array('f')
t = arr.array('f')
    # generate random numbers between 0-1
for _ in range(10):
    #make s
    value_s = random()
    value_s = -1 + (value_s * (0 +1))
    s.append(value_s)
    #make t given restriction with s
    value_t = - value_s - 1 - 0.01    
    t.append(value_t)
    #loop throygh s, and add t sch that it satisifes s+t < -1

#generate random momentum transfers t E (-1,0)
seed(1)
t_individual = arr.array('f')
# generate random numbers between 0-1
for _ in range(100):
    value = random()
    value = -1 + (value * (0 +1))
    t_individual.append(value)
    

# Define functions to be used in loss function

In [3]:
###Calculate M given array of z and J, for specific points (s,t)
import scipy
import scipy.integrate as integrate
from scipy.integrate import quad
import scipy.special


def Mst_Mts(s,t,z,J):
    return (M_withoutF(s,t,z,J) - M_withoutF(t,s,z,J))/N

def M_withoutF(s,t,z,J):   # use eq 2.6
    sum = 0
    A= []
    B = []
    for j in range(len(J)):
        for Z in z:
            a = 2/math.pi
            b = a*Z
            c = b*(1+Z*t)*(2+Z*t)
            d = c*np.array(P_J(1+2*Z*t, j))
            e = (1-Z*t)*(1+Z*(s+t))*s*(s+t)
            sum += d/e
            B.append(sum)
       # A.append(B)
    B= np.array(B)
    #A2 = np.array(A)
    return B

def P_J(x, J):
    #A =[]
    #for j in range(len(J)):
    B = scipy.special.lpmv(0, J, x)
    C = 16*math.pi*(2*J +1)
    D = C*B
    return D

def Gamma(z):
    return scipy.special.gamma(z)

def hypergeometric(a,b,c,d):
    return scipy.special.hyp2f1(a,b,c,d)

# def P_J(x, J):
#     dummy =  ((4*math.pi)**2)*(4 +(2*J) -3)
#     b = Gamma(J+1.0)
#     d0 = 1*b #gamma(1) = 1
#     d1 = math.pi*d0
#     D2 = (Gamma(1+J))/d1
#     d3 = dummy*D2
#     A =d3*hypergeometric(-J, J+1, 0, (1-x)/2)
    
#     return A
    #return (((4*math.pi)^(2)*(4 +2*J -3)*(Gamma(1+J)))/(math.pi*Gamma(1)*Gamma(J+1)))*hypergeometric(-J, J+1, 0, (1-x)/2)

    
    # def Gamma(z):
#     integrand = lambda x: x**(z-1)*math.exp(-x)
#     I = quad(integrand, 0, np.inf) 
#     print(I[0])
#     return I[0]

    
# def hypergeometric(a,b,c,d):
    
#     sum =0
#     for n in range (a):
#         sum +=  (-1**n) *scipy.special.comb(a, n) *  ((math.factorial(b))/(math.factorial(c)))* d**n
#     return sum


In [4]:
### K^(alpha)
###input = array of t_i, J_j and m_j
#Y sums over j, K sums over i.
def K_alpha(t, J, z):
    sum = 0
    k = []
    for j2 in J[0:]:
        for z2 in z[0:]:
            A = (-2*t/math.pi)
            B = A*z2
            C = B*(2 +t*z2)
            D = C*(P_J(1+2*z2*t, j2))
            sum = D
            k.append(sum/N)
            sum = 0
            
    
    return B
# add y in cost function

In [5]:
def K_Beta(J,z):
    sum =0
    A = []
    for j in J:
        for Z in z:
            sum += (-4/math.pi)*Z**(2)*P_J(1,j)
            A.append(sum/N)
        
    return A

# Use functions above to make arrays to be used in loss function

In [6]:
#Make Mst - mts array with s and t as scalars
# Making s and t tuples to loop over Mst_Mts function 
#X = tf.stack([z, J], axis=1)

a = np.transpose(s)
b = np.transpose(t)
M=[]
for i in s[0:]:
    for j in t[0:]:
        M.append(Mst_Mts(i,j,z,J))
        #break    
        
M= np.array(M)   
print(M.shape)


(100, 50)


In [156]:
K_A = []
for t1 in t_individual[0:]:
    
    value = K_alpha(t1,J,z)
    K_A.append(value)
    
K_A= np.array(K_A)
print(K_A.shape)

(100,)


In [154]:
K_B = K_Beta(J,z)
K_B= np.array(K_B)
print(K_B.shape)


(50,)


# Neural Network starts here

In [16]:
from __future__ import absolute_import, division, print_function
#from LossFunction_Functions import *  
#from LossFunction_Functions import M_withoutF, P_J, Gamma, hypergeometric, K_alpha, K_Beta

In [152]:
#X shape = [50,2]
#W1 = [2, 9]
#W2 = [9,1]
#b = [9]
# F = [50,1]

#X = np.transpose(X)
x = np.dstack((J, z))[0]
print(X.shape)
print(X)



ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 5 and the array at index 1 has size 10

In [43]:
import tensorflow as tf
import numpy as np
rng = np.random

In [44]:
# Parameters.
learning_rate = 0.01
training_steps = 1000
display_step = 50

In [119]:
G = 6.67430*(10**(-11))

def Loss_Function(F):
    #defie 8*PI*G 
    G = 6.67430*(10**(-11))
    c  = 8*math.pi*G
    
    alpha = 1
    beta = 10
    #remember squares
    #check if * is indeed dot produc for matriess
    print(F.shape)
    # first term
    Mf = tf.tensordot(M, F, axes=1)
    print(mf.shape)
    sum1 = 0
    for i in Mf[0:]:
        sum1 += i**2
  

    #second term
    Ka = tf.tensordot(K_A, F, axes = 1)
    sum2 = 0
    for j in Ka[0:]:
        sum2 += (j - c)**2
    
    #third term
    K_B = np.transpose(K_B)
    Kb = tf.tensordot(K_B, F, axes = 1)
    sum3 = Kb
  
    value = sum1 + alpha*sum2 +beta*sum3
    return value

In [139]:
# Weight and Bias, initialized randomly.
W1 = tf.Variable(rng.randn(2,9), name="weight", dtype=tf.float64, shape= [2,9])
W2 = tf.Variable(rng.randn(9,1), name="weight", dtype=tf.float64, shape= [9,1])
b = tf.Variable(rng.randn(9,), name="bias", dtype=tf.float64, shape= [9,])
# Linear regression (Wx + b).
def linear_regression(x):
    return W * x + b


# Stochastic Gradient Descent Optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

In [140]:
# Optimization process. 
def run_optimization():
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = linear_regression(X)
        loss = Loss_Function(pred) 
    # Compute gradients.
    gradients = g.gradient(loss, [W, b])
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, [W, b]))

In [141]:
# Run training for the given number of steps.
for step in range(1, training_steps + 1):
    # Run the optimization to update W and b values.
    run_optimization()
    print(step)
    if step % display_step == 0:
        pred = linear_regression(X)
        loss = Loss_Function(pred)
        print("step: %i, loss: %f, W: %f, b: %f" % (step, loss, W.numpy(), b.numpy()))


InvalidArgumentError: Incompatible shapes: [10,15] vs. [50,1] [Op:AddV2]

In [64]:
import matplotlib.pyplot as plt
# Graphic display
plt.plot(X, Y, 'ro', label='Original data')
plt.plot(X, np.array(W * X + b), label='Fitted line')
plt.legend()
plt.show()

NameError: name 'Y' is not defined

In [ ]:
%%capture
%run LossFunction_Functions .ipynb

In [150]:
SIZE =10
x1 = np.random.randint(0, 100, SIZE)
x2 = np.random.randint(0, 100, SIZE)
x = np.dstack((x1, x2))[0]
print(x)

[[80  7]
 [90 91]
 [40 13]
 [73 64]
 [46 77]
 [35 92]
 [79 53]
 [33 64]
 [60 95]
 [ 4 33]]
